# N-state and N-step Discrete Pricing

This program is completely generalizable - in the sense that one can enter number of states and number of time steps to get a holistic pricing model using the backward expectations iteration.

In [1]:
import numpy as np

In [2]:
# define the call option payoff

def call_option(ST, K):
    if ST > K:
        payoff = ST - K
    elif ST <= K:
        payoff = 0
    return payoff

In [167]:
# a mapping of the state space - a randomized function generates values for the states.

# the user has to choose a value for 'n'

n = 3
N = {}
for i in range(n):
    N[i] = np.round(abs(np.random.normal(2, 2)), 2)

In [168]:
# this is the state space - the possible movements are denoted by 'keys'
# and the possible values are the 'values'

N

{0: 1.75, 1: 0.18, 2: 3.98}

In [169]:
# this function generates a probability distribution over the states
# this is constructed so that the sum of all state probs is 1

P = {}
l = []
for i in range(n):
    l.append(np.round(abs(np.random.random(1)[0]), 3))
    
s = sum(l)
for j in range(len(l)):
    P[j] = np.round(l[j]/s, 2)

In [170]:
P

{0: 0.45, 1: 0.3, 2: 0.26}

In [171]:
# we draw the tree here - multiplying each node with 'n' 
# to generate successive nodes

def draw_tree(ts, S0, n):
    d1 = {}
    d2 = {}
    for t in range(ts):
        if t == 0:
            d1[t] = 1
        elif t > 0:
            d1[t] = d1[t-1]*n
        d2[t] = []

    d2[0].append(S0)
    
    return d1, d2

In [172]:
# arguments of this function call are:
# (time_step, today_stock_price, n_states)

D1, D2 = draw_tree(3, 100, n)
D1

{0: 1, 1: 3, 2: 9}

In [174]:
# simulating the values here - at each node multipying by 
# the state value and putting option payoffs in the last step

def simulate_values(ts, D_1, D_2, K, NN):
    for t in range(1, ts):
        for i in range(D_1[t-1]):
            for k in NN.keys():
                D_2[t].append(np.round(D_2[t-1][i]*NN[k], 2))
            
    D_2[len(D_2.keys())-1] = [np.round(call_option(x, K), 2) for x in D_2[len(D_2.keys())-1]]
    return D_2

In [175]:
# the 91 argument is merely the strike price -
# which the user can enter at will

DD2 = simulate_values(3, D1, D2, 91, N)

In [176]:
DD2

{0: [100],
 1: [175.0, 18.0, 398.0],
 2: [215.25, 0, 605.5, 0, 0, 0, 605.5, 0, 1493.04]}

In [177]:
# in this fucntion - we compute the backward iteration expectations
# note that expectations are computed using the dot product
# of the state probability vector and the node values

def backward_dynamic(dd, Q, n, r):
    h = {}
    probs = np.array(list(Q.values()))
    for t in sorted(list(dd.keys()), reverse=True):
        if t == 0:
            break

        arr = []
        for i in range(len(dd[t])):
            if i % n == 0:
                a = np.array(dd[t][i:i+n])
                E = np.exp(r)*(probs.dot(a))
                arr.append(round(E, 2))
            elif i % n != 0:
                pass

        h[t-1] = arr
        dd[t-1] = arr
        
    return h

In [178]:
OP = backward_dynamic(DD2, P, n, 0)
OP

{1: [254.29, 0.0, 660.67], 0: [286.2]}